In [12]:
import dill as pickle
import pandas as pd
import numpy as np
from shutil import copyfile
import sys
import os
from sklearn import *
sys.path.append("../fun")
from fun import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define run ID

In [13]:
settingsID = define_settings_id()

### Load data

In [14]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]

if "all_setting_results.pkl" in files:
    infile = open("all_setting_results.pkl",'rb')
    all_setting_results = pickle.load(infile)
    infile.close()
else:
    all_setting_results = {}

In [15]:
infile = open("../features/all_datasets_for_modeling.pkl",'rb')
all_datasets_for_modeling = pickle.load(infile)
infile.close()

In [16]:
infile = open("../features/build_train_test_for_model_parameters.pkl",'rb')
build_train_test_for_model_parameters= pickle.load(infile)
infile.close()

#### Metric de comparaison

In [17]:
def customscoring(y, y_pred):
    return metrics.mean_squared_error(y,y_pred)
myscoringfun = metrics.make_scorer(customscoring, greater_is_better=False)

#### Tout les modèles au sens large à tester

In [18]:
dic_of_models = {1:ensemble.RandomForestRegressor(),2:linear_model.Ridge()}

param_list_model_1 = [{"max_depth":np.random.randint(low=1,high=15,size=2), "random_state":0, "n_estimators":499,
                       "search_scoring" : myscoringfun,"search_cv":3, "search_n_jobs":2}]
param_list_model_2 = [{"random_state":1,"alpha":3, "split_test_size":0.45,"split_random_state":55},
                     {"random_state":999,"alpha":0.1}]
dic_of_hyparam = {1: param_list_model_1,
                  2: param_list_model_2}

# rajouter search_n_iter pour avoir de la randomsearchcv

#### Launch the big Run

In [19]:
results_of_the_run_settings = run_all_settings(all_datasets_for_modeling,
                                               dic_of_models,
                                               dic_of_hyparam,
                                               build_train_test_for_model_parameters,
                                               customscoring)

ID :  count_global_1.0 
 Feature Strate :  global 
 Target :  count 
 Dataset :  global 

Type :  GridSearchCV 
 Model :  RandomForestRegressor 
 Infos :  {'scoring': make_scorer(customscoring, greater_is_better=False), 'cv': 3, 'n_jobs': 2} 
 Features Number :  21 
 Features Names :  ['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'dataset', 'OHE_day_0', 'OHE_day_1', 'OHE_day_2', 'OHE_day_3', 'OHE_day_4', 'OHE_day_5', 'OHE_day_6', 'LE_day', 'LE_month', 'ORE_hour', 'ORE_weather'] 
 Nombre de modèles testés :  2 
 Score :  0.40037554290687044 
  
 * * * * * * * * * * * * * * * * * * * * * * * * *  

ID :  count_global_2.0 
 Feature Strate :  global 
 Target :  count 
 Dataset :  global 

Type :  holdout 
 Model :  Ridge 
 Infos :  {'test_size': 0.45, 'random_state': 55} 
 Features Number :  21 
 Features Names :  ['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'dataset', 'OHE_day_0'

#### Stocking and Saving Run Results

In [20]:
all_setting_results[settingsID] = results_of_the_run_settings
filename = 'all_setting_results.pkl'
outfile = open(filename,'wb')
pickle.dump(all_setting_results,outfile)
outfile.close()

#### Saving all notebooks written for this run in the results folder

In [21]:
folderpath = "results/"+settingsID
if not os.path.isdir(folderpath):
    os.mkdir("results/"+settingsID)

files_to_copy = listfiles_fromfolders([".","../features","../fun"])
copyfiles_to_folder(files_to_copy,folderpath)